In [1]:
import time, sys, os, glob, re, multiprocessing, psutil, tqdm
from pathlib import Path
import numpy as np

import conll


In [2]:
conlltxt = """# newdoc id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600
# sent_id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-0001
# newpar id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-p0001
# text = Elena's motorcycle tour through the region around Chernobyl has revived interest in one of the most serious nuclear disasters in history.
1-2	Elena's	_	_	_	_	_	_	_	_
1	Elena	Elena	PROPN	NNP	Number=Sing	4	nmod:poss	4:nmod:poss	_
2	's	's	PART	POS	_	1	case	1:case	_
3	motorcycle	motorcycle	NOUN	NN	Number=Sing	4	compound	4:compound	_
4	tour	tour	NOUN	NN	Number=Sing	11	nsubj	11:nsubj	_
5	through	through	ADP	IN	_	7	case	7:case	_
6	the	the	DET	DT	Definite=Def|PronType=Art	7	det	7:det	_
7	region	region	NOUN	NN	Number=Sing	4	nmod	4:nmod:through	_
8	around	around	ADP	IN	_	9	case	9:case	_
9	Chernobyl	Chernobyl	PROPN	NNP	Number=Sing	7	nmod	7:nmod:around	_
10	has	have	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	11	aux	11:aux	_
11	revived	revive	VERB	VBN	Tense=Past|VerbForm=Part	0	root	0:root	_
12	interest	interest	NOUN	NN	Number=Sing	11	obj	11:obj	_
13	in	in	ADP	IN	_	14	case	14:case	_
14	one	one	NUM	CD	NumType=Card	12	nmod	12:nmod:in	_
15	of	of	ADP	IN	_	20	case	20:case	_
16	the	the	DET	DT	Definite=Def|PronType=Art	20	det	20:det	_
17	most	most	ADV	RBS	_	18	advmod	18:advmod	_
18	serious	serious	ADJ	JJ	Degree=Pos	20	amod	20:amod	_
19	nuclear	nuclear	ADJ	JJ	Degree=Pos	20	amod	20:amod	_
20	disasters	disaster	NOUN	NNS	Number=Plur	14	nmod	14:nmod:of	_
21	in	in	ADP	IN	_	22	case	22:case	_
22	history	history	NOUN	NN	Number=Sing	14	nmod	14:nmod:in	SpaceAfter=No
23	.	.	PUNCT	.	_	11	punct	11:punct	_
"""

In [3]:
tree=conll.conll2tree(conlltxt)

In [4]:
print(type(tree))
kk = tree[1]['gov'].keys()
print(kk)
for k in kk:
    print(k)
    
kl = list(kk)
print(kl)
print("\n",tree)

<class 'conll.Tree'>
dict_keys([4])
4
[4]

 # newdoc id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600
# newpar id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-p0001
# sent_id = weblog-typepad.com_ripples_20040407125600_ENG_20040407_125600-0001
# text = Elena's motorcycle tour through the region around Chernobyl has revived interest in one of the most serious nuclear disasters in history.
1	Elena	Elena	PROPN	NNP	Number=Sing	4	nmod:poss	4:poss	_
2	's	's	PART	POS	_	1	case	1:case	_
3	motorcycle	motorcycle	NOUN	NN	Number=Sing	4	compound	4:compound	_
4	tour	tour	NOUN	NN	Number=Sing	11	nsubj	11:nsubj	_
5	through	through	ADP	IN	_	7	case	7:case	_
6	the	the	DET	DT	Definite=Def|PronType=Art	7	det	7:det	_
7	region	region	NOUN	NN	Number=Sing	4	nmod	4:through	_
8	around	around	ADP	IN	_	9	case	9:case	_
9	Chernobyl	Chernobyl	PROPN	NNP	Number=Sing	7	nmod	7:around	_
10	has	have	AUX	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	11	aux	11:aux	_
11	revived	r

In [5]:
def headOfList(idlist,tree):
    hdlist = []
    if len(idlist):
        for i in idlist:
            for hd in tree[i]['gov'].keys():
                if hd != -1: hdlist.append(hd)
    return hdlist

In [6]:
headlist = []
for i in range(1, len(tree)+1):
    for hd in tree[i]['gov'].keys():
        headlist.append(hd)    
print(headlist)

hhl = headOfList(range(1, len(tree)+1),tree)
print(hhl)

[4, 1, 4, 11, 7, 7, 4, 9, 7, 11, 0, 11, 14, 12, 20, 20, 18, 20, 20, 14, 22, 14, 11]
[4, 1, 4, 11, 7, 7, 4, 9, 7, 11, 0, 11, 14, 12, 20, 20, 18, 20, 20, 14, 22, 14, 11]


In [7]:
def haveCousin(hdset, tree):
    hdset.remove(0) #we are going to find head of headset while 0 haven't head
    #find head of headset, an id can appears more than once if there are multiple heads in one stage 
    hdofhdList = headOfList(hdset, tree)
    hdofhdSet = set(hdofhdList) #eliminate duplication
    return len(hdofhdList)!= len(set(hdofhdSet)), hdofhdSet

In [8]:
def treeHeight(tree):
    """
    find tree height with different head id. For each 'stage' of tree, we compte only one head. 
    if one node is a head, it means there is one more stage for its son (dependent). 
    while in one stage, maybe there are more than one head(who has branche), so we need delete the others
    """
    height = -1
    headset = set(headOfList(tree.keys(),tree)) #all head id in current tree
    print("head id of current tree:", headset)
    
    branch = 0
    cousin = True #if there are cousins in headset
    while(cousin):
        cousin, headset = haveCousin(headset, tree) 
        print("headset == ", headset)
        branch += 1
        print(branch)
        
        if(not cousin):
            height = len(headset)+ branch
            
    return height

In [9]:
ht = treeHeight(tree)
print("tree height = ", ht)

head id of current tree: {0, 1, 4, 7, 9, 11, 12, 14, 18, 20, 22}
headset ==  {0, 4, 7, 11, 12, 14, 20}
1
headset ==  {0, 4, 11, 12, 14}
2
headset ==  {0, 11, 12}
3
headset ==  {0, 11}
4
tree height =  6


In [10]:
tree.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

##### node height

In [11]:
def getHeadId(idx, tree): #put to private
    for hd in tree[idx]['gov'].keys():#hd is unique for each node
        return hd

In [12]:
#dep length
#for one node, find the depth by track back to the root

def depLength(nodeId, tree):
    length = 1 #every node have at least length 1 to the node 0
    head = getHeadId(nodeId, tree)
    while(head != 0):
        idx = head
        head = getHeadId(idx, tree)
        length += 1
    return length

In [13]:
assert(depLength(11,tree) == 1)
assert(depLength(12,tree) == 2)
assert(depLength(4,tree) == 2)
assert(depLength(1,tree) == 3)
assert(depLength(2,tree) == 4)
assert(depLength(21,tree) == 5)
assert(depLength(17,tree) == 6)

??? Then stock deplength of nodes by tag of token for each language and find the average value 

#### empty node

In [14]:
testE = """# sent_id = weblog-blogspot.com_dakbangla_20041028153019_ENG_20041028_153019-0002
# text = Over nine dozen temples that had served the Hindu population have been destroyed, with some used as building material and others as urinals.
1	Over	over	ADV	RB	_	3	advmod	3:advmod	_
2	nine	nine	NUM	CD	NumType=Card	3	compound	3:compound	_
3	dozen	dozen	NOUN	NN	Number=Sing	4	nummod	4:nummod	_
4	temples	temple	NOUN	NNS	Number=Plur	13	nsubj:pass	7:nsubj|13:nsubj:pass	_
5	that	that	PRON	WDT	PronType=Rel	7	nsubj	4:ref	_
6	had	have	AUX	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	7	aux	7:aux	_
7	served	serve	VERB	VBN	Tense=Past|VerbForm=Part	4	acl:relcl	4:acl:relcl	_
8	the	the	DET	DT	Definite=Def|PronType=Art	10	det	10:det	_
9	Hindu	hindu	ADJ	JJ	Degree=Pos	10	amod	10:amod	_
10	population	population	NOUN	NN	Number=Sing	7	obj	7:obj	_
11	have	have	AUX	VBP	Mood=Ind|Tense=Pres|VerbForm=Fin	13	aux	13:aux	_
12	been	be	AUX	VBN	Tense=Past|VerbForm=Part	13	aux:pass	13:aux:pass	_
13	destroyed	destroy	VERB	VBN	Tense=Past|VerbForm=Part	0	root	0:root	SpaceAfter=No
14	,	,	PUNCT	,	_	13	punct	13:punct	_
15	with	with	SCONJ	IN	_	17	mark	17:mark	_
16	some	some	DET	DT	_	17	nsubj	17:nsubj|22:nsubj	_
17	used	use	VERB	VBN	Tense=Past|VerbForm=Part	13	advcl	13:advcl:with	_
18	as	as	ADP	IN	_	20	case	20:case	_
19	building	building	NOUN	NN	Number=Sing	20	compound	20:compound	_
20	material	material	NOUN	NN	Number=Sing	17	obl	17:obl:as	_
21	and	and	CCONJ	CC	_	22	cc	22:cc|22.1:cc	_
22	others	other	NOUN	NNS	Number=Plur	17	conj	13:advcl:with|17:conj:and|22.1:nsubj	_
22.1	used	use	VERB	VBN	Tense=Past|VerbForm=Part	_	_	13:advcl:with|17:conj:and	CopyOf=17
23	as	as	ADP	IN	_	24	case	24:case	_
24	urinals	urinal	NOUN	NNS	Number=Plur	22	orphan	22.1:obl:as	SpaceAfter=No
25	.	.	PUNCT	.	_	13	punct	13:punct	_
"""

In [15]:
treeE=conll.conll2tree(testE)
treeE.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 22.1, 23, 24, 25])

In [17]:
treeE[22.1]

{'CopyOf': '17',
 'Tense': 'Past',
 'VerbForm': 'Part',
 'id': 22.1,
 't': 'used',
 'lemma': 'use',
 'tag': 'VERB',
 'xpos': 'VBN',
 'gov': {-1: '_'},
 'egov': {'13': 'with', '17': 'and'}}

In [16]:
hht = treeHeight(treeE)
print("tree height = ", hht)

head id of current tree: {0, 3, 4, 7, 10, 13, 17, 20, 22, 24}
headset ==  {0, 4, 7, 13, 17, 22}
1
headset ==  {0, 17, 4, 13}
2
headset ==  {0, 13}
3
headset ==  {0}
4
tree height =  5


#### multiword token

In [18]:
testMulti = """# sent_id = weblog-blogspot.com_dakbangla_20041028153019_ENG_20041028_153019-0008
# text = As Pakistan's president had been telling the United States he was "forced" into taking a hawkish line on India precisely by the likes of Rahman, this was an embarrassment.
1	As	as	SCONJ	IN	_	7	mark	7:mark	_
2-3	Pakistan's	_	_	_	_	_	_	_	_
2	Pakistan	Pakistan	PROPN	NNP	Number=Sing	4	nmod:poss	4:nmod:poss	_
3	's	's	PART	POS	_	2	case	2:case	_
4	president	president	NOUN	NN	Number=Sing	7	nsubj	7:nsubj	_
5	had	have	AUX	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	7	aux	7:aux	_
6	been	be	AUX	VBN	Tense=Past|VerbForm=Part	7	aux	7:aux	_
7	telling	tell	VERB	VBG	Tense=Pres|VerbForm=Part	33	advcl	33:advcl:as	_
8	the	the	DET	DT	Definite=Def|PronType=Art	10	det	10:det	_
9	United	United	ADJ	NNP	Degree=Pos	10	amod	10:amod	_
10	States	State	PROPN	NNPS	Number=Plur	7	obj	7:obj	_
11	he	he	PRON	PRP	Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs	14	nsubj:pass	14:nsubj:pass	_
12	was	be	AUX	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	14	aux:pass	14:aux:pass	_
13	"	"	PUNCT	``	_	14	punct	14:punct	SpaceAfter=No
14	forced	force	VERB	VBN	Tense=Past|VerbForm=Part|Voice=Pass	7	ccomp	7:ccomp	SpaceAfter=No
15	"	"	PUNCT	''	_	14	punct	14:punct	_
16	into	into	SCONJ	IN	_	17	mark	17:mark	_
17	taking	take	VERB	VBG	VerbForm=Ger	14	advcl	14:advcl:into	_
18	a	a	DET	DT	Definite=Ind|PronType=Art	20	det	20:det	_
19	hawkish	hawkish	ADJ	JJ	Degree=Pos	20	amod	20:amod	_
20	line	line	NOUN	NN	Number=Sing	17	obj	17:obj	_
21	on	on	ADP	IN	_	22	case	22:case	_
22	India	India	PROPN	NNP	Number=Sing	17	obl	17:obl:on	_
23	precisely	precisely	ADV	RB	_	26	advmod	26:advmod	_
24	by	by	ADP	IN	_	26	case	26:case	_
25	the	the	DET	DT	Definite=Def|PronType=Art	26	det	26:det	_
26	likes	likes	NOUN	NN	Number=Sing	14	obl	14:obl:by	_
27	of	of	ADP	IN	_	28	case	28:case	_
28	Rahman	Rahman	PROPN	NNP	Number=Sing	26	nmod	26:nmod:of	SpaceAfter=No
29	,	,	PUNCT	,	_	33	punct	33:punct	_
30	this	this	PRON	DT	Number=Sing|PronType=Dem	33	nsubj	33:nsubj	_
31	was	be	AUX	VBD	Mood=Ind|Number=Sing|Person=3|Tense=Past|VerbForm=Fin	33	cop	33:cop	_
32	an	a	DET	DT	Definite=Ind|PronType=Art	33	det	33:det	_
33	embarrassment	embarrassment	NOUN	NN	Number=Sing	0	root	0:root	SpaceAfter=No
34	.	.	PUNCT	.	_	33	punct	33:punct	_
"""

In [19]:
treeM=conll.conll2tree(testMulti)
treeM.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

In [20]:
print("tree height = ", treeHeight(treeM))

head id of current tree: {0, 33, 2, 4, 7, 10, 14, 17, 20, 22, 26, 28}
headset ==  {0, 33, 4, 7, 14, 17, 26}
1
headset ==  {0, 33, 14, 7}
2
headset ==  {0, 33, 7}
3
tree height =  6
